In [6]:
import tensorflow as tf
import json
import random 
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from riotwatcher import LolWatcher, ApiError, RiotWatcher
import numpy as np
import sklearn
import pandas as pd
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import random
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [7]:
infile = open("data_2.json","r")
dataset = json.load(infile)
infile.close()

In [8]:
df = pd.DataFrame()
columns=["W_TOP","W_JUNGLE","W_MIDDLE","W_BOTTOM","W_SUPPORT","W_BARON","W_BARON_FIRST","W_DRAGON","W_DRAGON_FIRST","W_CHAMPS","W_CHAMPS_FIRST","W_INHIB","W_INHIB_FIRST","W_RIFT","W_RIFT_FIRST","W_TOWER","W_TOWER_FIRST",
                           "L_TOP","L_JUNGLE","L_MIDDLE","L_BOTTOM","L_SUPPORT","L_BARON","L_BARON_FIRST","L_DRAGON","L_DRAGON_FIRST","L_CHAMPS","L_CHAMPS_FIRST","L_INHIB","L_INHIB_FIRST","L_RIFT","L_RIFT_FIRST","L_TOWER","L_TOWER_FIRST"]

In [9]:
df = pd.DataFrame()
w_top =[]
w_jg = []
w_mid = []
w_bot = []
w_sup = []

l_top = []
l_jg = []
l_mid = []
l_bot = []
l_sup = []

w_drag = []
w_drag_first = []
w_baron = []
w_baron_first = []
w_champs = []
w_champs_first = []
w_inhib = []
w_inhib_first = []
w_rift = []
w_rift_first = []
w_tower = []
w_tower_first = []

l_drag = []
l_drag_first = []
l_baron = []
l_baron_first = []
l_champs = []
l_champs_first = []
l_inhib = []
l_inhib_first = []
l_rift = []
l_rift_first = []
l_tower = []
l_tower_first = []

i = 0
for match in dataset:
    temp_players = []
    if match["info"]["gameMode"] == "CLASSIC" and match["info"]["gameType"] == "MATCHED_GAME" and len(match["metadata"]["participants"]) == 10 and len(match["info"]["participants"]) == 10:
        teams = match["info"]["teams"]
        for team in teams:
            if team["win"] == True:
                w_baron_first.append(int(team["objectives"]["baron"]["first"]))
                w_baron.append(int(team["objectives"]["baron"]["kills"]))
                w_drag_first.append(int(team["objectives"]["dragon"]["first"]))
                w_drag.append(int(team["objectives"]["dragon"]["kills"]))
                w_champs_first.append(int(team["objectives"]["champion"]["first"]))
                w_champs.append(int(team["objectives"]["champion"]["kills"]))
                w_inhib_first.append(int(team["objectives"]["inhibitor"]["first"]))
                w_inhib.append(int(team["objectives"]["inhibitor"]["kills"]))
                w_rift_first.append(int(team["objectives"]["riftHerald"]["first"]))
                w_rift.append(int(team["objectives"]["riftHerald"]["kills"]))
                w_tower_first.append(int(team["objectives"]["tower"]["first"]))
                w_tower.append(int(team["objectives"]["tower"]["kills"]))

            if team["win"] == False:
                l_baron_first.append(int(team["objectives"]["baron"]["first"]))
                l_baron.append(int(team["objectives"]["baron"]["kills"]))
                l_drag_first.append(int(team["objectives"]["dragon"]["first"]))
                l_drag.append(int(team["objectives"]["dragon"]["kills"]))
                l_champs_first.append(int(team["objectives"]["champion"]["first"]))
                l_champs.append(int(team["objectives"]["champion"]["kills"]))
                l_inhib_first.append(int(team["objectives"]["inhibitor"]["first"]))
                l_inhib.append(int(team["objectives"]["inhibitor"]["kills"]))
                l_rift_first.append(int(team["objectives"]["riftHerald"]["first"]))
                l_rift.append(int(team["objectives"]["riftHerald"]["kills"]))
                l_tower_first.append(int(team["objectives"]["tower"]["first"]))
                l_tower.append(int(team["objectives"]["tower"]["kills"]))

        for player in match["info"]["participants"]:
            if player["win"] == True and player["teamPosition"] == "TOP":
                w_top.append(player["championId"])
            elif player["win"] == True and player["teamPosition"] == "JUNGLE":
                w_jg.append(player["championId"])
            elif player["win"] == True and player["teamPosition"] == "MIDDLE":
                w_mid.append(player["championId"])
            elif player["win"] == True and player["teamPosition"] == "BOTTOM":
                w_bot.append(player["championId"])
            elif player["win"] == True and player["teamPosition"] == "UTILITY":
                w_sup.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "TOP":
                l_top.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "JUNGLE":
                l_jg.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "MIDDLE":
                l_mid.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "BOTTOM":
                l_bot.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "UTILITY":
                l_sup.append(player["championId"])
            else:
                temp_players.append(player["championId"])
        if len(temp_players) != 0:
            for t_player in temp_players:
                player_pos = [w_top, w_jg, w_mid, w_bot, w_sup, l_top, l_jg, l_mid, l_bot, l_sup]
                for position in player_pos:
                    if len(position) == i:
                        position.append(t_player)
        i+=1
pos = [w_top, w_jg, w_mid, w_bot, w_sup, w_baron, w_baron_first, w_drag, w_drag_first, w_champs, w_champs_first, w_inhib, w_inhib_first, w_rift, w_rift_first, 
       w_tower, w_tower_first, l_top, l_jg, l_mid, l_bot, l_sup, l_baron, l_baron_first, l_drag, l_drag_first, l_champs, l_champs_first, l_inhib, l_inhib_first,
        l_rift, l_rift_first, l_tower, l_tower_first]
for i in range(len(pos)):
    # print(len(pos[i]))
    df.insert(loc=i,column=columns[i],value=pos[i])

In [10]:
#print(df.iloc[:,0:7])
#print(df.iloc[:,17:25])
print(df.iloc[:,[6,8,10,12,14,16]])
print(df.iloc[:,[23,25,27,29,31,33]])
print(df.iloc[:,6].to_list().count(1)/len(df))
print(df.iloc[:,23].to_list().count(1)/len(df))

       W_BARON_FIRST  W_DRAGON_FIRST  W_CHAMPS_FIRST  W_INHIB_FIRST  \
0                  0               0               0              0   
1                  0               1               0              0   
2                  1               0               1              1   
3                  1               0               0              1   
4                  1               0               1              0   
...              ...             ...             ...            ...   
45585              0               0               1              0   
45586              0               1               1              1   
45587              0               1               0              0   
45588              1               1               0              1   
45589              1               1               1              1   

       W_RIFT_FIRST  W_TOWER_FIRST  
0                 0              0  
1                 1              0  
2                 1              1  

In [11]:
win_team = []
lose_team = []
# for i in range(len(df)):
#     win_team.append(list(df.iloc[i][0:5]))

# print(win_team)
win_team = df.iloc[:,0:5].values.tolist()
obj_w = df.iloc[:,[6,8,10,12,14,16]].values.tolist()
lose_team = df.iloc[:,17:22].values.tolist()
obj_l = df.iloc[:,[23,25,27,29,31,33]].values.tolist()
y = []
x = []
for i in range(len(win_team)):
    swap =  random.randint(0,1)
    y.append(swap)
    if swap == 0:
        x.append(obj_w[i] + obj_l[i])
        #x.append(win_team[i] + [obj_w[i]] + lose_team[i] + [obj_l[i]])
    else:
        x.append(obj_l[i] + obj_w[i])
        #x.append(lose_team[i] + [obj_l[i]] + win_team[i] + [obj_w[i]])

print(len(x))
X_train,X_val,y_train,y_val = train_test_split(x,y, test_size=0.15)

45590


In [12]:
infile = open("data3.json","r")
dataset = json.load(infile)
infile.close()
columns=["W_TOP","W_JUNGLE","W_MIDDLE","W_BOTTOM","W_SUPPORT","W_BARON","W_BARON_FIRST","W_DRAGON","W_DRAGON_FIRST","W_CHAMPS","W_CHAMPS_FIRST","W_INHIB","W_INHIB_FIRST","W_RIFT","W_RIFT_FIRST","W_TOWER","W_TOWER_FIRST",
                           "L_TOP","L_JUNGLE","L_MIDDLE","L_BOTTOM","L_SUPPORT","L_BARON","L_BARON_FIRST","L_DRAGON","L_DRAGON_FIRST","L_CHAMPS","L_CHAMPS_FIRST","L_INHIB","L_INHIB_FIRST","L_RIFT","L_RIFT_FIRST","L_TOWER","L_TOWER_FIRST"]

df_test = pd.DataFrame()
w_top =[]
w_jg = []
w_mid = []
w_bot = []
w_sup = []

l_top = []
l_jg = []
l_mid = []
l_bot = []
l_sup = []

w_drag = []
w_drag_first = []
w_baron = []
w_baron_first = []
w_champs = []
w_champs_first = []
w_inhib = []
w_inhib_first = []
w_rift = []
w_rift_first = []
w_tower = []
w_tower_first = []

l_drag = []
l_drag_first = []
l_baron = []
l_baron_first = []
l_champs = []
l_champs_first = []
l_inhib = []
l_inhib_first = []
l_rift = []
l_rift_first = []
l_tower = []
l_tower_first = []

i = 0
for match in dataset:
    temp_players = []
    if match["info"]["gameMode"] == "CLASSIC" and match["info"]["gameType"] == "MATCHED_GAME" and len(match["metadata"]["participants"]) == 10 and len(match["info"]["participants"]) == 10:
        teams = match["info"]["teams"]
        for team in teams:
            if team["win"] == True:
                w_baron_first.append(int(team["objectives"]["baron"]["first"]))
                w_baron.append(int(team["objectives"]["baron"]["kills"]))
                w_drag_first.append(int(team["objectives"]["dragon"]["first"]))
                w_drag.append(int(team["objectives"]["dragon"]["kills"]))
                w_champs_first.append(int(team["objectives"]["champion"]["first"]))
                w_champs.append(int(team["objectives"]["champion"]["kills"]))
                w_inhib_first.append(int(team["objectives"]["inhibitor"]["first"]))
                w_inhib.append(int(team["objectives"]["inhibitor"]["kills"]))
                w_rift_first.append(int(team["objectives"]["riftHerald"]["first"]))
                w_rift.append(int(team["objectives"]["riftHerald"]["kills"]))
                w_tower_first.append(int(team["objectives"]["tower"]["first"]))
                w_tower.append(int(team["objectives"]["tower"]["kills"]))

            if team["win"] == False:
                l_baron_first.append(int(team["objectives"]["baron"]["first"]))
                l_baron.append(int(team["objectives"]["baron"]["kills"]))
                l_drag_first.append(int(team["objectives"]["dragon"]["first"]))
                l_drag.append(int(team["objectives"]["dragon"]["kills"]))
                l_champs_first.append(int(team["objectives"]["champion"]["first"]))
                l_champs.append(int(team["objectives"]["champion"]["kills"]))
                l_inhib_first.append(int(team["objectives"]["inhibitor"]["first"]))
                l_inhib.append(int(team["objectives"]["inhibitor"]["kills"]))
                l_rift_first.append(int(team["objectives"]["riftHerald"]["first"]))
                l_rift.append(int(team["objectives"]["riftHerald"]["kills"]))
                l_tower_first.append(int(team["objectives"]["tower"]["first"]))
                l_tower.append(int(team["objectives"]["tower"]["kills"]))

        for player in match["info"]["participants"]:
            if player["win"] == True and player["teamPosition"] == "TOP":
                w_top.append(player["championId"])
            elif player["win"] == True and player["teamPosition"] == "JUNGLE":
                w_jg.append(player["championId"])
            elif player["win"] == True and player["teamPosition"] == "MIDDLE":
                w_mid.append(player["championId"])
            elif player["win"] == True and player["teamPosition"] == "BOTTOM":
                w_bot.append(player["championId"])
            elif player["win"] == True and player["teamPosition"] == "UTILITY":
                w_sup.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "TOP":
                l_top.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "JUNGLE":
                l_jg.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "MIDDLE":
                l_mid.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "BOTTOM":
                l_bot.append(player["championId"])
            elif player["win"] == False and player["teamPosition"] == "UTILITY":
                l_sup.append(player["championId"])
            else:
                temp_players.append(player["championId"])
        if len(temp_players) != 0:
            for t_player in temp_players:
                player_pos = [w_top, w_jg, w_mid, w_bot, w_sup, l_top, l_jg, l_mid, l_bot, l_sup]
                for position in player_pos:
                    if len(position) == i:
                        position.append(t_player)
        i+=1
pos = [w_top, w_jg, w_mid, w_bot, w_sup, w_baron, w_baron_first, w_drag, w_drag_first, w_champs, w_champs_first, w_inhib, w_inhib_first, w_rift, w_rift_first, 
       w_tower, w_tower_first, l_top, l_jg, l_mid, l_bot, l_sup, l_baron, l_baron_first, l_drag, l_drag_first, l_champs, l_champs_first, l_inhib, l_inhib_first,
        l_rift, l_rift_first, l_tower, l_tower_first]
for i in range(len(pos)):
    # print(len(pos[i]))
    df_test.insert(loc=i,column=columns[i],value=pos[i])

In [13]:
win_team = []
lose_team = []
# for i in range(len(df)):
#     win_team.append(list(df.iloc[i][0:5]))

# print(win_team)
win_team = df_test.iloc[:,0:5].values.tolist()
obj_w = df_test.iloc[:,[6,8,10,12,14,16]].values.tolist()
lose_team = df_test.iloc[:,17:22].values.tolist()
obj_l = df_test.iloc[:,[23,25,27,29,31,33]].values.tolist()
y_test = []
X_test = []
for i in range(len(win_team)):
    swap =  random.randint(0,1)
    y_test.append(swap)
    if swap == 0:
        X_test.append(obj_w[i] + obj_l[i])
        #X_test.append(win_team[i] + [obj_w[i]] + lose_team[i] + [obj_l[i]])
    else:
        X_test.append(obj_l[i] + obj_w[i])
        #X_test.append(lose_team[i] + [obj_l[i]] + win_team[i] + [obj_w[i]])

In [14]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

knn_clf = KNeighborsClassifier(n_neighbors=10)
knn_clf.fit(X_train, y_train)
y_pred = knn_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

forest_clf = RandomForestClassifier(n_estimators=35)
forest_clf.fit(X_train,y_train)
y_pred = forest_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))

0.8672438672438673
[[635  88]
 [ 96 567]]
0.8585858585858586
[[620 103]
 [ 93 570]]
0.8679653679653679
[[635  88]
 [ 95 568]]


In [15]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Reshape(target_shape=(12,),input_shape=(12,)),
        tf.keras.layers.Dense(units = 30,activation='sigmoid'),
        tf.keras.layers.Dense(units = 30,activation='sigmoid'),
        tf.keras.layers.Dense(units=30,activation='sigmoid'),
        tf.keras.layers.Dense(units = 30,activation='sigmoid'),
        tf.keras.layers.Dense(units = 30,activation='sigmoid'),
        tf.keras.layers.Dense(units = 30,activation='sigmoid'),
        tf.keras.layers.Dense(units = 30,activation='sigmoid'),
        tf.keras.layers.Dense(units=2,activation='softmax')
    ])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join('ckpt', "{epoch:02d}-{val_loss:.4f}.hdf5"),
        monitor = 'val_loss',
        verbose = 0,
        save_best_only = True,
        save_weights_only = False,
        mode = 'auto',
        save_freq='epoch',
        options=None,
        initial_value_threshold=None,
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=20,
        verbose=1,
        mode='auto',
        min_delta=0.0001,
        cooldown=0,
        min_lr=0,
        ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=50,
        verbose=1,
        mode='auto',
        baseline=None,
        restore_best_weights=True,
    )
]

model.compile(optimizer="adam",loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
history = model.fit(X_train,y_train,batch_size=50,callbacks = callbacks, epochs=400,validation_data=(X_val,y_val))


Epoch 1/400
776/776 [==============================] - 4s 4ms/step - loss: 0.4263 - accuracy: 0.7850 - val_loss: 0.2804 - val_accuracy: 0.9110 - lr: 0.0010
Epoch 2/400
776/776 [==============================] - 3s 4ms/step - loss: 0.2843 - accuracy: 0.9061 - val_loss: 0.2817 - val_accuracy: 0.9076 - lr: 0.0010
Epoch 3/400
776/776 [==============================] - 2s 3ms/step - loss: 0.2843 - accuracy: 0.9048 - val_loss: 0.2884 - val_accuracy: 0.9041 - lr: 0.0010
Epoch 4/400
776/776 [==============================] - 3s 3ms/step - loss: 0.2845 - accuracy: 0.9046 - val_loss: 0.2796 - val_accuracy: 0.9101 - lr: 0.0010
Epoch 5/400
776/776 [==============================] - 3s 4ms/step - loss: 0.2842 - accuracy: 0.9049 - val_loss: 0.2792 - val_accuracy: 0.9080 - lr: 0.0010
Epoch 6/400
776/776 [==============================] - 3s 4ms/step - loss: 0.2839 - accuracy: 0.9045 - val_loss: 0.2789 - val_accuracy: 0.9082 - lr: 0.0010
Epoch 7/400
776/776 [==============================] - 3s 4ms/st

In [16]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)

mat = sklearn.metrics.confusion_matrix(y_test,y_pred)
print(mat)
print((mat[0][0]+mat[1][1])/len(y_pred))
print(sklearn.metrics.precision_recall_fscore_support(y_test,y_pred))

44/44 [==============================] - 0s 2ms/step
[[635  88]
 [ 95 568]]
0.8679653679653679
(array([0.86986301, 0.86585366]), array([0.87828492, 0.85671192]), array([0.87405368, 0.86125853]), array([723, 663], dtype=int64))


In [17]:
y_pred = forest_clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))
print(sklearn.metrics.precision_recall_fscore_support(y_test,y_pred))

0.8679653679653679
[[635  88]
 [ 95 568]]
(array([0.86986301, 0.86585366]), array([0.87828492, 0.85671192]), array([0.87405368, 0.86125853]), array([723, 663], dtype=int64))


In [18]:
import joblib
joblib.dump(forest_clf,'pred_models/comp_rf/forest_clf.pkl')

['pred_models/comp_rf/forest_clf.pkl']

In [19]:
#clf = joblib.load('pred_models/comp_rf/forest_clf.pkl')
y_pred = clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)
print(sklearn.metrics.confusion_matrix(y_test,y_pred))
print(sklearn.metrics.precision_recall_fscore_support(y_test,y_pred))

0.8672438672438673
[[635  88]
 [ 96 567]]
(array([0.86867305, 0.86564885]), array([0.87828492, 0.85520362]), array([0.87345254, 0.86039454]), array([723, 663], dtype=int64))


KeyError: ''